# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Gould  ->  A. Gould  |  ['A. Gould']
F. Walter  ->  F. Walter  |  ['F. Walter']
K. Lee  ->  K. Lee  |  ['K. Lee']
Arxiv has 79 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2405.16857


extracting tarball to tmp_2405.16857... done.


/tmp/ipykernel_2295/1211882699.py:51: LatexWarning: 2405.16857 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2405.17000


...

F. Walter  ->  F. Walter  |  ['F. Walter']


Found 138 bibliographic references in tmp_2405.17000/LSBD-MHONGOOSE.bbl.
syntax error in line 375: '=' expected
Retrieving document from  https://arxiv.org/e-print/2405.17095


extracting tarball to tmp_2405.17095...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:411: LatexWarning: Latex injecting: '49772corr.bbl' from 'tmp_2405.17095/49772corr.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.17000-b31b1b.svg)](https://arxiv.org/abs/2405.17000) | **MHONGOOSE discovery of a gas-rich low-surface brightness galaxy in the Dorado Group**  |
|| F. M. Maccagni, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-05-28*|
|*Comments*| *18 pages, 14 figures, 5 tables; accepted for publication in A&A*|
|**Abstract**|            We present the discovery of a low-mass gas-rich low-surface brightness galaxy in the Dorado Group, at a distance of 17.7 Mpc. Combining deep MeerKAT 21-cm observations from the MeerKAT HI Observations of Nearby Galactic Objects: Observing Southern Emitters (MHONGOOSE) survey with deep photometric images from the VST Early-type Galaxy Survey (VEGAS) we find a stellar and neutral atomic hydrogen (HI) gas mass of $M_\star = 2.23\times10^6$ M$_\odot$ and $M_{\rm HI}=1.68\times10^6$ M$_\odot$, respectively. This low-surface brightness galaxy is the lowest mass HI detection found in a group beyond the Local Universe ($D\gtrsim 10$ Mpc). The dwarf galaxy has the typical overall properties of gas-rich low surface brightness galaxies in the Local group, but with some striking differences. Namely, the MHONGOOSE observations reveal a very low column density ($\sim 10^{18-19}$ cm$^{-2}$) HI disk with asymmetrical morphology possibly supported by rotation and higher velocity dispersion in the centre. There, deep optical photometry and UV-observations suggest a recent enhancement of the star formation. Found at galactocentric distances where in the Local Group dwarf galaxies are depleted of cold gas (at $390$ projected-kpc distance from the group centre), this galaxy is likely on its first orbit within the Dorado group. We discuss the possible environmental effects that may have caused the formation of the HI disk and the enhancement of star formation, highlighting the short-lived phase (a few hundreds of Myr) of the gaseous disk, before either SF or hydrodynamical forces will deplete the gas of the galaxy.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.17095-b31b1b.svg)](https://arxiv.org/abs/2405.17095) | **Large-amplitude transverse MHD waves prevailing in the H$\alpha$ chromosphere of a solar quiet region revealed by MiHI integrated field spectral observations**  |
|| J. Chae, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-05-28*|
|*Comments*| *accepted for publication in A&A*|
|**Abstract**|            The investigation of plasma motions in the solar chromosphere is crucial for understanding the transport of mechanical energy from the interior of the Sun to the outer atmosphere and into interplanetary space. We report the finding of large-amplitude oscillatory transverse motions prevailing in the non-spicular Halpha chromosphere of a small quiet region near the solar disk center. The observation was carried out on 2018 August 25 with the Microlensed Hyperspectral Imager (MiHI) installed as an extension to the spectrograph at the Swedish Solar Telescope (SST). MiHi produced high-resolution Stokes spectra of the Halpha line over a two-dimensional array of points (sampled every 0.066 arcsec on the image plane) every 1.33 s for about 17 min. We extracted the Dopple-shift-insensitive intensity data of the line core by applying a bisector fit to Stoke I line profiles. From our time-distance analysis of the intensity data, we find a variety of transverse motions with velocity amplitudes of up to 40 km/s in fan fibrils and tiny filaments. In particular, in the fan fibrils, large-amplitude transverse MHD waves were seen to occur with a mean velocity amplitude of 25 km/s and a mean period of 5.8 min, propagating at a speed of 40 km/s. These waves are nonlinear and display group behavior. We estimate the wave energy flux in the upper chromosphere at 3 x 10^6 erg cm^-2 s^-1. Our results contribute to the advancement of our understanding of the properties of transverse MHD waves in the solar chromosphere.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.16857-b31b1b.svg)](https://arxiv.org/abs/2405.16857) | **KMT-2023-BLG-2669: Ninth Free-floating Planet Candidate with $\theta_{\rm E}$ measurements**  |
|| Y. K. Jung, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2024-05-28*|
|*Comments*| *17 pages, 5 figures, submitted to AJ*|
|**Abstract**|            We report a free-floating planet (FFP) candidate identified from the analysis of the microlensing event KMT-2023-BLG-2669. The lensing light curve is characterized by a short duration $(\lesssim 3\,{\rm days})$ and a small amplitude $(\lesssim 0.7\,{\rm mag})$. From the analysis, we find the Einstein timescale of $t_{\rm E} \backsimeq 0.33\,{\rm days}$ and the Einstein radius of $\theta_{\rm E} \backsimeq 4.41\,{\mu}{\rm as}$. These measurements enable us to infer the lens mass as $M = 8\,M_{\oplus} (\pi_{\rm rel} / 0.1\,{\rm mas})^{-1}$, where $\pi_{\rm rel}$ is the relative lens-source parallax. The inference implies that the lens is a sub-Neptune- to Saturn-mass object depending on its unknown distance. This is the ninth isolated planetary-mass microlens with $\theta_{\rm E} < 10\,{\mu}{\rm as}$, which (as shown by \citealt{gould22}) is a useful threshold for a FFP candidate. We conduct extensive searches for possible signals of a host star in the light curve, but find no strong evidence for the host. We discuss the possibility of using late-time high-resolution imaging to probe for possible hosts.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2405.17000.md
    + _build/html/tmp_2405.17000/./rgb_cont.png
    + _build/html/tmp_2405.17000/./rgba_lsbd_mom0mom1_v5_referee.png
    + _build/html/tmp_2405.17000/./TF-mcG-MgasMstarAAAA_referee.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\fluxhi}{{S_{\rm 1.4  GHz}}}$
$\newcommand{\fluxco}{{S_{\rm CO   (2-1)}}}$
$\newcommand{\pow}{{\log P_{1.4   \rm GHz}}}$
$\newcommand{\matHI}{{\rm H {\hskip 0.02cm \tt I}}}$
$\newcommand{\CO}{\element[]CO}$
$\newcommand{\couno}{\element[][12]CO (1--0) }$
$\newcommand{\codue}{\element[][12]CO (2--1) }$
$\newcommand{\cotre}{{\element[][12]CO(3--2)} }$
$\newcommand{\brgamma}{Br\gamma}$
$\newcommand{\sisei}{[Si {\small VI}]}$
$\newcommand{\Htwozero}{H_{ 2} 0-0 S(0,1,2,5)}$
$\newcommand{\Htwouno}{H_{ 2} 1-0 S(0,1,2,3)}$
$\newcommand{\Htwotwo}{H_{ 2} 2-1 S(1,3)}$
$\newcommand{\Htwoz}{H_{ 2} 1-0 S(0)}$
$\newcommand{\Htwoi}{H_{ 2} 1-0 S(1)}$
$\newcommand{\Htwod}{H_{ 2} 1-0 S(2)}$
$\newcommand{\Htwot}{H_{ 2} 1-0 S(3)}$
$\newcommand{\Htwodz}{H_{ 2} 2-1 S(1)}$
$\newcommand{\Htwodt}{H_{ 2} 2-1 S(3)}$
$\newcommand{\mhtwo}{{M (H_{  2})}}$
$\newcommand{\kms}{ km~s^{-1}}$
$\newcommand{\ms}{ m~s^{-1}}$
$\newcommand{\ergs}{ erg~s^{-1}}$
$\newcommand{\ergscm}{ erg s^{-1} cm^{-2}}$
$\newcommand{\kmsMp}{km s^{-1}~Mpc^{-1}}$
$\newcommand{\Jyb}{Jy beam^{-1}}$
$\newcommand{\Jykms}{Jy km s^{-1}}$
$\newcommand{\mJyb}{mJy beam^{-1}}$
$\newcommand{\mJy}{mJy}$
$\newcommand{\whz}{W~Hz^{-1}}$
$\newcommand{\msun}{{{\rm M}_\odot}}$
$\newcommand{\msunyr}{{{\rm M}_\odot yr^{-1}}}$
$\newcommand{\lsun}{{L_\odot}}$
$\newcommand{\cmsq}{cm^{-2}}$
$\newcommand{\pcdue}{pc^{-2}}$
$\newcommand{\tspin}{T_{\rm spin}}$
$\newcommand{\atms}{atoms cm^{-2}}$
$\newcommand{\um }{\mum}$
$\newcommand{\eg}{\mbox{e.g.}}$
$\newcommand{\ie}{\mbox{i.e.}}$
$\newcommand{\etal}{{\sl et al.}}$
$\newcommand{\sauron}{{\texttt{SAURON}}}$
$\newcommand{\atlas}{{ATLAS^{\rm 3D}}}$
$\newcommand{\mgas}{{MeerGas}}$
$\newcommand{\mhon}{{MHONGOOSE}}$
$\newcommand{\lsb}{\mbox LSB-D}$
$\newcommand{\forn}{\mbox Fornax~A}$
$\newcommand{\cena}{\mbox Centaurus~A}$
$\newcommand{\nfi}{\mbox NGC~1566}$
$\newcommand{\pl}{{\em{Planck}}}$
$\newcommand{\meer}{{MeerKAT}}$
$\newcommand{\syn}{\tt{SYNAGE++}}$
$\newcommand{\cara}{{\tt CARACal}}$
$\newcommand{\vsys}{{v_{\rm sys}} }$
$\newcommand{\vrot}{{v_{\rm rot}} }$
$\newcommand{\MHILB}{{M_{\rm HI}/L_B}}$
$\newcommand{\nhi}{{N_{\rm H {\hskip 0.02cm \tt I}}}}$
$\newcommand{\mhi}{{M_{\rm H {\hskip 0.02cm \tt I}}}}$
$\newcommand{\alphac}{{\alpha_{\rm CO}}}$
$\newcommand{\MLB}{M/L_B}$
$\newcommand{\NB}[1]{\textcolor{red}{\textbf{[NB: #1]}}}$
$\newcommand{\HI}{\ion{H}{I}}$
$\newcommand{\halpha}{\ion{H\alpha}}$
$\newcommand{\hbeta}{\ion{H\beta}}$
$\newcommand{\nad}{\ion{NaI}~D}$
$\newcommand{\OIst}{[\ion{O}{I}]\lambda6300}$
$\newcommand{\OIsts}{[\ion{O}{I}]\lambda6364}$
$\newcommand{\OIIIqn}{[\ion{O}{III}]\lambda4959}$
$\newcommand{\OIIIfs}{[\ion{O}{III}]\lambda5007}$
$\newcommand{\NIIscq}{[\ion{N}{II}]\lambda6548}$
$\newcommand{\NIIsco}{[\ion{N}{II}]\lambda6583}$
$\newcommand{\SIIuno}{[\ion{S}{II}]\lambda6716}$
$\newcommand{\SIIdue}{[\ion{S}{II}]\lambda6731}$
$\newcommand{\SIIdoublet}{[\ion{S}{II}]\lambda\lambda6716,6731}$
$\newcommand{\OI}{[\ion{O}{I}]}$
$\newcommand{\OIII}{[\ion{O}{III}]}$
$\newcommand{\NII}{[\ion{N}{II}]}$
$\newcommand{\SII}{[\ion{S}{II}]}$
$\newcommand{\HII}{H{ \small II}}$
$\newcommand{\OIV}{[O{ \small IV}]}$
$\newcommand{\HII}{H{ \small II}}$
$\newcommand{\NII}{[N{ \small II}]}$
$\newcommand{\OIV}{[O{ \small IV}]}$
$\newcommand{\Htwo}{H_{ 2}}$</div>



<div id="title">

# MHONGOOSE discovery of a gas-rich low-surface brightness galaxy in the Dorado Group

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.17000-b31b1b.svg)](https://arxiv.org/abs/2405.17000)<mark>Appeared on: 2024-05-28</mark> -  _18 pages, 14 figures, 5 tables; accepted for publication in A&A_

</div>
<div id="authors">

F. M. Maccagni, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present the discovery of a low-mass gas-rich low-surface brightness galaxy in the Dorado Group, at a distance of 17.7 Mpc. Combining deep MeerKAT 21-cm observations from the MeerKAT $\HI$ Observations of Nearby Galactic Objects: Observing Southern Emitters (MHONGOOSE) survey with deep photometric images from the VST Early-type Galaxy Survey (VEGAS) we find a stellar and neutral atomic hydrogen ( $\HI$ ) gas mass of $M_\star = 2.23\times10^6$  $\msun$ and $\mhi$ $=1.68\times10^6$  $\msun$ , respectively. This low-surface brightness galaxy is the lowest mass $\HI$ detection found in a group beyond the Local Universe ( $D\gtrsim 10$ Mpc). The dwarf galaxy has the typical overall properties of gas-rich low surface brightness galaxies in the Local group, but with some striking differences. Namely, the MHONGOOSE observations reveal a very low column density ( $\sim 10^{18-19}$  $\cmsq$ ) $\HI$ disk with asymmetrical morphology possibly supported by rotation and higher velocity dispersion in the centre. There, deep optical photometry and UV-observations suggest a recent enhancement of the star formation. Found at galactocentric distances where in the Local Group dwarf galaxies are depleted of cold gas (at $390$ projected-kpc distance from the group centre), this galaxy is likely on its first orbit within the Dorado group. We discuss the possible environmental effects that may have caused the formation of the $\HI$ disk and the enhancement of star formation, highlighting the short-lived phase (a few hundreds of Myr) of the gaseous disk, before either SF or hydrodynamical forces will deplete the gas of the galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2405.17000/./rgb_cont.png" alt="Fig11" width="100%"/>

**Figure 11. -** Primary beam corrected flux-density $\HI$ emission detected by $\meer$ within a $1.5\times 1.5$ deg$^2$ field centred on $\nfi$(the imaged f.o.v. is marked by the dashed grey circle), overlaid on the DECaLS optical image in the g,r,z filters. The $\HI$ emission is a composite of flux-density maps derived from the multi-resolution datacubes. The resolutions shown by coloured contours are $12$\arcsec$\times10$\arcsec(red), $25$\arcsec$\times18$\arcsec(green), $32$\arcsec$\times23$\arcsec(blue), $65$\arcsec$\times64$\arcsec(purple), $94$\arcsec$\times92$\arcsec(yellow-green). For resolutions between $\sim 90$\arcsec and $25$\arcsec two contours are shown, $\ie$, $\nhi$$=3\sigma\times2^n$ with $n = 0, 2$. The $12$\arcsec$\times10$\arcsec resolution has further increasing contours ($n=0,2,4$...). The PSF of the multi-resolution maps are shown in the bottom-right corner with the same colour coding. Details on the $\HI$ properties of the sources in the field are given in Table \ref{tab:masses}.  (*fig:fullFieldCont*)

</div>
<div id="div_fig2">

<img src="tmp_2405.17000/./rgba_lsbd_mom0mom1_v5_referee.png" alt="Fig12" width="100%"/>

**Figure 12. -** * Left Panel*: DeCaLS optical image in the $g,r,z$ filters overlaid with the $\HI$ contours from the $25$\arcsec$\times18$\arcsec datacube used for the analysis in this paper. Levels increase as $4.6\times10^{18}\times 2^n$ $\cmsq$(n=0,1,2,3), where the first contour marks the mean $S/N=3$ detection limit. The PSF is shown in the bottom-right corner. Note the $\HI$ overlay in color with also the $32\times23$ $\arcsec$ map. * Right Panel*: Velocity field of the $\HI$ gas in $\lsb$. The systemic velocity ($v_{\rm sys} = 1214$ $\kms$) is marked by the thick black iso-velocity contour, the other contour levels are shown in the bottom-left corner. The dashed and dashed-dotted lines mark the directions of the major ($141^\circ$) and minor axes ($51^\circ$) of the $\HI$ disk, respectively (*fig:mom01*)

</div>
<div id="div_fig3">

<img src="tmp_2405.17000/./TF-mcG-MgasMstarAAAA_referee.png" alt="Fig6" width="100%"/>

**Figure 6. -** * Top panel*: Gas mass versus stellar mass for a sample of galaxies in the Local Universe.  LITTLE-THINGS, SPARC and THINGS galaxies are marked by triangles (in orange, grey and blue, respectively). Local low surface brightness galaxies (LSBGs) from \citet[][]{McGaugh:2011,McGaugh:2017,mcGaugh:2021} are in green, and UDGs in cyan. $\lsb$ is marked by the red circle. The dashed line marks the $1:1$ linear relation. * Bottom panel*: The baryonic Tully–Fisher relation of Local Group dwarf and spiral galaxies \citep[dashed line][]{mcGaugh:2021}. Symbols are as in the top panel. For the Local LSBGs sample, pressure supported dwarfs (plotted against their velocity dispersion, $\sigma_v$) are marked by light green squares, while green triangles mark rotationally-supported galaxies, plotted against their rotational velocity $v_{\rm rot}$. (*fig:TF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.17000"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

79  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
